In [1]:
import polars as pl
import os

In [2]:
# URL_2023 = 'https://servicos.dpf.gov.br/dadosabertos/SINARM_CSV/OCORRENCIAS/OCORRENCIAS_ate_2023.csv'
URL_2024 = 'https://servicos.dpf.gov.br/dadosabertos/SINARM_CSV/OCORRENCIAS/OCORRENCIAS_2024.csv'

In [3]:
# df_primary = pl.read_csv(URL_2023,separator=',', encoding='ISO-8859-1')
df  = pl.read_csv(URL_2024,separator=';', encoding='ISO-8859-1')

In [4]:
df.head()

ANO_OCORRENCIA,MES_OCORRENCIA,UF,MUNICIPIO,ESPECIE_ARMA,MARCA_ARMA,CALIBRE_ARMA,TIPO_OCORRENCIA,MAIS_1000_MIL_HAB,TOTAL
i64,i64,str,str,str,str,str,str,str,i64
2024,1,"""AC""","""ACRELÂNDIA …","""Espingarda …","""ROSSI (AMADEO ROSSI S.A.) …","""28 …","""Furto de Arma de Fogo …","""N""",1
2024,1,"""AC""","""ACRELÂNDIA …","""Pistola …","""TAURUS ARMAS S.A. …","""9x19mm …","""Furto de Arma de Fogo …","""N""",1
2024,1,"""AC""","""RIO BRANCO …","""Pistola …","""TAURUS ARMAS S.A. …",""".380 ACP …","""Furto de Arma de Fogo …","""S""",1
2024,1,"""AC""","""RIO BRANCO …","""Pistola …","""TAURUS ARMAS S.A. …",""".45 ACP …","""Apostilada no Exercito …","""S""",1
2024,1,"""AC""","""RIO BRANCO …","""Pistola …","""TAURUS ARMAS S.A. …","""9 mm …","""Recuperação de Arma de Fogo …","""S""",1


In [5]:
df = df.rename({col: col.lower() for col in df.columns})
df = df.with_columns(pl.col('ano_ocorrencia').cast(pl.Int32), pl.col('mes_ocorrencia').cast(pl.Int32) )

In [6]:
connUrl = f'postgresql://{os.environ['POSTGRE_USER']}:{os.environ['POSTGRE_PASSWORD']}@{os.environ['POSTGRE_HOST']}:5432/{os.environ['POSTGRE_DATABASE']}'

In [7]:
df.write_database('raw.ocorrencias',connUrl, if_table_exists='append', engine="adbc")

5026